# MuSCAT2 $target $night TFOP analysis

In [ ]:
%pylab inline

In [ ]:
from muscat2ta.tfopanalysis import TFOPAnalysis

## Plot the field

![]($field)

## Initialise TFOP analysis

The TFOP analysis is done with the `muscat2ta.tfopanalysis.TFOPAnalysis` class that contains all the necessary methods for the TFOP data reduction and analysis.

`TFOPAnalysis` is initialised with

 - `target`: target name exactly as in the `MuSCAT2` catalog
 - `date`: Observing date in the format yymmdd
 - `tid`: target ID
 - `cids`: list of IDs of the useful comparison stars to include into the comparison star optimisation. *Note:* The final set of comparison stars will be a subset of these stars. The analysis is relatively robust against bad comparison stars, but each comparison star adds `npb` free parameters to the optimisation.
 
and has a set of optional arguments that can be used to fine-tune the analysis

 - `passbands ['g','r','i','z_s']`: passbands to include into the analysis.
 - `aperture_limits [(0, inf)]`: a (min, max) tuple of aperture ID limits for the aperture optimisation.
 - `use_opencl [False]`: use OpenCL (some parts of the computations will be done with the GPU) if `True`.
 - `with_transit [True]`: include a transit model into the light curve model if `True`.
 - `contamination [None]`: allow the flux to be contaminated by an unresolved star if `True`.
 - `radius_ratio ['achromatic']`: `chromatic` for passband dependent radius ratios and `achromatic` for passband independent radius ratios.
 - `npop [200]`: number of parameter vectors in the optimisation and MCMC.
 - `toi [None]`: `TOI` object with the ExoFOP TOI information.
 - `klims [(0.005, 0.25)]`: radius ratio limits for the transit model. The upper limit needs to be increased if modelling an EB.

In [ ]:
ta = TFOPAnalysis('$target', '$night', 0, CIDS, use_opencl=False)

## Light curve cleanup
### Plot the target and the n brightest stars

These plots show the raw absolute light curves for the target and the brightest stars in the field. The plots will not be saved, but help when choosing the reference stars and deciding if the light curve needs to be trimmed.

In [ ]:
ta.plot_raw_light_curves(ylim=(0.95, 1.05));

**Light curve trimming:**

If the light curve contains spans of bad data, they can be trimmed away usin `ta.cut()`. The method will remove the data **between** `tstart` and `tend`. If you want to cut all the data from beginning till `START_TIME`, you can call the method as

    ta.lpf.cut(tend=START_TIME, apply=False)

if you want to cut all the data after `END_TIME`, you can call the method as

    ta.lpf.cut(tstart=END_TIME, apply=False)
    
and if you want to remove the data between times `T1` and `T2`, you can call the method as 
   
    ta.lpf.cut(tstart=T1, tend=t2, apply=False)

*Note:* You need to set `apply=True` to actually apply the outlier removal. By default the command only plots what would be removed if `apply` would be `True`.

In [ ]:
ta.lpf.cut(tstart=-inf, tend=-inf, apply=False)

**Outlier removal:** The pipeline comes with several ways to remove outliers, of which `ta.lpf.apply_normalized_limits(aid, lower, upper, apply, plot, npoly, iterations)` is the most user friendly. The method removes points with relative normalised flux smaller than `lower` and larger than `upper`.

*Note:* You need to set `apply=True` to actually apply the outlier removal. By default the command only plots what would be removed if `apply` would be `True`.

In [ ]:
ta.apply_normalized_limits(4, lower=0.9, upper=1.1, apply=False, plot=True, npoly=2, iterations=5)

**Downsampling:** the light curves can be downsampled (binned) to 30-60 second exposures in most cases without compromising the quality of the analysis. This can speed up the analysis significantly, and is especially recommended with bright TESS targets.

In [ ]:
ta.downsample(60)

## Create the basic TFOP files

### Plot possible blends

Plot the raw fluxes from all the stars that are within 2.5 arcmin from the target. The plot is saved to the final output directory.

These plots are important for TFOP and will be saved in the result directory. The plots show the raw light curves for the target star and all the stars around it within a given radius (unbinned and binned). The plots also show the expected times for the transit start, centre, and end (with their uncertainties), and the expected transit signal with depth corresponding to the depth that would be observed if the transit would be on the blending star (in reality we'd expect to see even deeper signal, because these plots assume blending only between the target and the possible blend, while in reality the blend is blended with multiple sources). The plots show the flux ratio between the target and the possible contaminant on the top-right corner. 

`ta.plot_possible_blends` requires two arguments

- `cid:` comparison star index. Should be a bright (non-saturated) star outside a 2 arcmin radius of the target star.
- `aid:` aperture index. The aperture should be large enough to capture the flux of the brightest star within 2 arcmin from the target.

In [ ]:
ta.plot_possible_blends(cid=CID, aid=AID)

### Plot covariates

Plot the covariates for all the passbands and save the figure in the final output directory.

In [ ]:
ta.plot_covariates()

### Export the raw data

Export the photometry in .csv files that are saved in the final output directory.

In [ ]:
ta.export_tables()

## Transit modelling

### Global optimisation

Transit modelling starts with a global optimisation run using `TFOPAnalysis.optimize`. This step optimises the set of comparison stars, target and comparison star apertues, a baseline model using the covariates and (optional) time variability modelled as Legendre polynomials, and (optional) transit model.

The global optimisation can be done semi-interactively. First run the optimisation (run the cell below) with a small number of iterations (200-500) and plot the fit (run the second cell below). Next, run the optimisation again (run the cell below), and the optimiser will continue from where it left, and again check the optimisation by plotting the fit. If everything looks fine (the optimiser seems to be converging to a sensible solution), do a longer optimisation run (1000-2000 iterations), plot the fit, and continue to sampling. If the optimisation stops before it reaches the number of iterations, it has converged to a point, and you can continue to sampling

In [ ]:
ta.optimize(1000)

In [ ]:
fig, axs = ta.plot_fit('de')

### MCMC sampling

The MCMC sampling of the posterior is done using `TFOPAnalysis.sample`. First, the set of comparison stars and target and comparison apertures should be frozen using `TFOPAnalysis.freeze_photometry`. This reduces the number of free parameters, and significantly improves the MCMC sampling efficiency. We'll also follow the freezing with a small optimisation step.

The sampling can be done the same way as optimisation. Run the sampler for some (500-1000) iterations and plot the fit. Also check the chain population using `TFOPAnalysis.plot_mcmc_chains` to see how well the sampler is behaving. Do a longer final MCMC run after everything looks ok, and continue.

In [ ]:
ta.freeze_photometry()

In [ ]:
ta.optimize(500)

In [ ]:
fig, axs = ta.plot_fit('de')

In [ ]:
ta.sample(niter=500, repeats=15)

In [ ]:
ta.plot_final_fit()

## Save the MCMC sampling and the reduced light curve

In [ ]:
ta.save()

In [ ]:
ta.save_fits()

In [ ]:
ta.finalize()